In [1]:
from sqlalchemy.ext import *

In [2]:
print("test line")

test line


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

DB_URL = os.getenv("PG_DATABASE_URL")

In [5]:
# In a new cell in your notebook

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
import os
from dotenv import load_dotenv

load_dotenv()
DB_URL = os.getenv("PG_DATABASE_URL")

# Standard engine setup
engine = create_engine(DB_URL)

# 1. Create an Automap Base
# This is a special Base class that can reflect the database schema.
Base = automap_base()

# 2. Reflect the tables
# This is the key step. SQLAlchemy connects to the DB,
# reads the schema of all tables, and prepares to create classes.
Base.prepare(autoload_with=engine)

# 3. Access the mapped classes
# The classes are now available under Base.classes.<table_name>
# Let's assign them to variables for easier use.
Users = Base.classes.users
Address = Base.classes.addresses

# Setup the session, same as before
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

print("Successfully mapped existing tables: ", Base.classes.keys())
# This will print something like: dict_keys(['users', 'addresses'])

Successfully mapped existing tables:  ['users', 'addresses']


## **CRUD Operations**

In [ ]:
# Open a session
db = SessionLocal()

# --- READ ---
print("\n--- READING DATA ---")
# Let's find a user that already exists in the database
# We'll assume a user with username 'Al Amin' is already there.
from sqlalchemy import select

stmt = select(Users).where(Users.username == 'Al Amin')
user_to_update = db.scalars(stmt).first()

if user_to_update:
    print(f"Found existing user: {user_to_update}")
    # SQLAlchemy's automap usually figures out relationships!
    # Let's try to access the addresses.
    if hasattr(user_to_update, 'address_collection'): # automap often names it `<table>_collection`
         print(f"Found addresses: {user_to_update.address_collection}")

else:
    print("User 'Al Amin' not found in the existing database.")

# --- UPDATE ---
print("\n--- UPDATING DATA ---")
if user_to_update:
    print(f"Old balance: {user_to_update.balance}")
    user_to_update.balance += 50  # Give them a raise
    db.commit() # The session tracks the change and sends an UPDATE statement
    db.refresh(user_to_update)
    print(f"New balance after commit: {user_to_update.balance}")
else:
    print("Cannot update a user that wasn't found.")

# --- CREATE ---
print("\n--- CREATING DATA ---")
# Let's add a new user that doesn't exist yet
new_user = Users(username='squidward', age=45, balance=300)
db.add(new_user)
db.commit()
print(f"Created a new user with ID: {new_user.id}")

# --- DELETE ---
print("\n--- DELETING DATA ---")
# Now let's delete the user we just created
user_to_delete = db.get(Users, new_user.id) # .get() is great for finding by primary key
if user_to_delete:
    db.delete(user_to_delete)
    db.commit()
    print(f"User '{user_to_delete.username}' has been deleted.")

# Close the session when done
db.close()


--- READING DATA ---
Found existing user: <sqlalchemy.ext.automap.users object at 0x000001C8EA373C20>

--- UPDATING DATA ---
Old balance: 120.5
New balance after commit: 170.5

--- CREATING DATA ---
Created a new user with ID: 4

--- DELETING DATA ---
User 'squidward' has been deleted.
